In [ ]:
#-- Importar librerías --#
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
from collections import Counter
import math

In [ ]:
#--     Definiciones de parámetros     --#
#--     y^2 = x^3 + a*x + b mod(p)     --#
#-- G es un punto de la curva con orden r --#
a = 0
b = 7
p = 433
G = (6, 400)
k_secret = 371

In [ ]:
#-- Operaciones de curvas elípticas --#
def inv_mod(x, p):
    if x % p == 0:
        raise ZeroDivisionError("no inverse for 0")
    return pow(x, p-2, p)

def point_add(P, Q, a, p):
    if P is None:
        return Q
    if Q is None:
        return P
    (x1, y1) = P
    (x2, y2) = Q
    if x1 == x2 and (y1 + y2) % p == 0:
        return None
    if P != Q:
        dx = (x2 - x1) % p
        lam = ((y2 - y1) * inv_mod(dx, p)) % p
    else:
        if y1 % p == 0:
            return None
        num = (3 * x1 * x1 + a) % p
        den = (2 * y1) % p
        lam = (num * inv_mod(den, p)) % p
    x3 = (lam * lam - x1 - x2) % p
    y3 = (lam * (x1 - x3) - y1) % p
    return (x3, y3)

def scalar_mult(k, P, a, p):
    if P is None or k == 0:
        return None
    Q = None
    N = P
    kk = k
    while kk > 0:
        if kk & 1:
            Q = point_add(Q, N, a, p)
        N = point_add(N, N, a, p)
        kk >>= 1
    return Q

R = scalar_mult(k_secret, G, a, p)

def order_of_point(P):
    if P is None:
        return 1
    Q = P
    r = 1
    while True:
        if Q is None:
            return r
        Q = point_add(Q, P, a, p)
        r += 1
        if r > p+10:  # safety
            return r

rG = order_of_point(G)

print("Parámetros: p={}, a={}, b={}".format(p,a,b))
print("G =", G, "R =", R, "order(G) =", rG)

In [ ]:
#-- Pre computación clásica --#
n_s = 5
n_t = 5
M_s = 2**n_s
M_t = 2**n_t

point_list = []
mapping_st_to_id = {}

for s in range(M_s):
    for t in range(M_t):
        P = point_add(scalar_mult(s, G, a, p), scalar_mult(t, R, a, p), a, p)
        if P not in point_list:
            point_list.append(P)
        pid = point_list.index(P)
        mapping_st_to_id[(s, t)] = pid

num_points = len(point_list)
m_id = math.ceil(math.log2(max(1, num_points)))
if m_id == 0:
    m_id = 1

print(f"Combinaciones s,t = {M_s}x{M_t} = {M_s*M_t}. Distintos puntos encontrados: {num_points}. bits para id ={m_id}")

In [ ]:
#-- Armado del circuito --#
s_reg = QuantumRegister(n_s, name='s')
t_reg = QuantumRegister(n_t, name='t')
id_reg = QuantumRegister(m_id, name='pid')
anc = QuantumRegister(max(0, (n_s+n_t)-2), name='anc') if (n_s+n_t) > 2 else None
c_id = ClassicalRegister(m_id, name='cid')
c_s = ClassicalRegister(n_s, name='cs')
c_t = ClassicalRegister(n_t, name='ct')

qc = QuantumCircuit(s_reg, t_reg, id_reg, *( [anc] if anc is not None else [] ), c_id, c_s, c_t)

qc.h(s_reg)
qc.h(t_reg)

def apply_pattern_controlled_set(circ, s_val, t_val, target_qubit_idx, target_bit_value):
    controls = []
    s_bits = format(s_val, f"0{n_s}b")
    t_bits = format(t_val, f"0{n_t}b")
    
    for i, b in enumerate(s_bits):
        q = s_reg[i]
        controls.append(q)
        if b == '0':
            circ.x(q)
            
    for i, b in enumerate(t_bits):
        q = t_reg[i]
        controls.append(q)
        if b == '0':
            circ.x(q)
            
    target = id_reg[target_qubit_idx]
    if target_bit_value == 1:
        if anc is not None and len(controls) > 2:
            circ.mcx(controls, target, anc)
        else:
            circ.mcx(controls, target)
            
    for i, b in enumerate(s_bits):
        q = s_reg[i]
        if b == '0':
            circ.x(q)
            
    for i, b in enumerate(t_bits):
        q = t_reg[i]
        if b == '0':
            circ.x(q)

for s in range(M_s):
    for t in range(M_t):
        pid = mapping_st_to_id[(s, t)]
        for bit_index in range(m_id):
            bit_val = (pid >> (m_id - 1 - bit_index)) & 1
            if bit_val == 1:
                apply_pattern_controlled_set(qc, s, t, bit_index, 1)
                
qc.measure(id_reg, c_id)

def qft(circ, reg):
    n = len(reg)
    for j in range(n):
        circ.h(reg[j])
        for k in range(j+1, n):
            angle = math.pi / (2 ** (k-j))
            circ.cp(angle, reg[k], reg[j])
    for i in range(n//2):
        circ.swap(reg[i], reg[n-1-i])

qft(qc, s_reg)
qft(qc, t_reg)

qc.measure(s_reg, c_s)
qc.measure(t_reg, c_t)

In [ ]:
#-- Ejecución del circuito --#
service = QiskitRuntimeService()
n_qubits = qc.num_qubits
backend = service.least_busy(
    operational=True,
    simulator=False,
    min_num_qubits=n_qubits
)
print(f"Usando backend físico: {backend.name}")

pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
tqc = pm.run(qc)

sampler = SamplerV2(backend)
job = sampler.run([tqc], shots=2048).result()

print(job)
cs_counts = job[0].data.cs.get_counts()
ct_counts = job[0].data.ct.get_counts()
cs_bitstrings = job[0].data.cs.get_bitstrings()
ct_bitstrings = job[0].data.ct.get_bitstrings()

print(len(job[0].data.cs.get_bitstrings()))
print(len(job[0].data.ct.get_bitstrings()))

In [ ]:
#-- Post procesado clásico --#
all_shots = []
for i in range(len(cs_bitstrings)):
    shot = cs_bitstrings[i] + ct_bitstrings[i]
    all_shots.append(shot)
print(all_shots[:10])

pairs = []
for i in range(len(cs_bitstrings)):
    cs = cs_bitstrings[i]
    ct = ct_bitstrings[i]
    u = int(cs, 2)
    v = int(ct, 2)
    pairs.append((u, v))
print("Ejemplo primeros 10 pares (u, v):", pairs[:10])

def recover_k_from_pairs(pairs, r, G, R, a, p):
    candidates = []
    for (u, v) in pairs:
        if v % r == 0:
            continue
        try:
            v_inv = pow(v, -1, r)
        except ValueError:
            continue
        k_cand = (-u * v_inv) % r
        print("- ", k_cand, end='')
        if scalar_mult(k_cand, G, a, p) == R:
            candidates.append(k_cand)
            
    if not candidates:
        return None, {}
        
    freq = Counter(candidates)
    topk, _ = freq.most_common(1)[0]
    return topk, freq

k_cand, freq = recover_k_from_pairs(pairs, rG, G, R, a, p)

print("cantidad de qubits =", qc.num_qubits)
print("Candidato k =", k_cand)
print("Distribución de candidatos:", freq.most_common(10))